In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "C:\\Users\\farha\\Desktop\\Py\\Pencil_shading"

# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.1,allow_growth=True)
# session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    #Restrict Tensorflow to only allocate 6gb of memory on the first GPU
   try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
   except RuntimeError as e:
       #virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [5]:
# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(
        data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 3061 images belonging to 5 classes.
Found 3061 images belonging to 5 classes.


In [6]:
# Create GoogLeNet model
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [7]:
# Freeze all layers except the top ones
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train model
history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=10)

Epoch 1/10
95/95 [==============================] - 738s 8s/step - loss: 1.5139 - accuracy: 0.6824 - val_loss: 0.5101 - val_accuracy: 0.8477
Epoch 2/10
95/95 [==============================] - 427s 5s/step - loss: 0.6357 - accuracy: 0.7963 - val_loss: 0.4402 - val_accuracy: 0.8579
Epoch 3/10
95/95 [==============================] - 425s 5s/step - loss: 0.5779 - accuracy: 0.8082 - val_loss: 0.3951 - val_accuracy: 0.8645
Epoch 4/10
95/95 [==============================] - 422s 4s/step - loss: 0.5215 - accuracy: 0.8227 - val_loss: 0.3762 - val_accuracy: 0.8773
Epoch 5/10
95/95 [==============================] - 421s 4s/step - loss: 0.4904 - accuracy: 0.8389 - val_loss: 0.3751 - val_accuracy: 0.8809
Epoch 6/10
95/95 [==============================] - 389s 4s/step - loss: 0.4312 - accuracy: 0.8580 - val_loss: 0.2831 - val_accuracy: 0.8990
Epoch 7/10
95/95 [==============================] - 388s 4s/step - loss: 0.4186 - accuracy: 0.8462 - val_loss: 0.3223 - val_accuracy: 0.8819
Epoch 8/10
95

In [10]:
model.save("GoogleNet.h5")